## Bibliotecas Utilizadas

- Json: 
    - Permite a codificação e decodificação de dados no formato JSON (JavaScript Object Notation):
- loggin:
    - Oferece um framework flexível para geração de logs:
- Pathlib: 
    - Fornece ferramentas para trabalhar com caminhos de diretórios (pastas):

In [ ]:
import json
import logging
from pathlib import Path

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def convert_coco_to_yolo(size, box):
    """
    Converte a bounding box (caixa delimitadora) do formato COCO para o formato YOLO.
    
    Argumentos:
        size (tuple): Uma tupla (largura, altura) representando as dimensões da imagem.
        box (list): Uma lista [x_superior_esquerdo, y_superior_esquerdo, largura, altura] representando uma bounding box no formato COCO.
    
    Retorna:
        tupla: Uma tupla (x_centro, y_centro, largura, altura) representando a bounding box no formato YOLO.
            - x_centro (float): Coordenada X do centro da caixa, normalizada pela largura da imagem.
            - y_centro (float): Coordenada Y do centro da caixa, normalizada pela altura da imagem.
            - largura (float): Largura da caixa, normalizada pela largura da imagem.
            - altura (float): Altura da caixa, normalizada pela altura da imagem.
    """
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2] / 2.0) * dw
    y = (box[1] + box[3] / 2.0) * dh
    w = box[2] * dw
    h = box[3] * dh
    return (x, y, w, h)

def process_annotations(dataset_path):
    """
    Processa anotações no formato COCO e as converte para o formato YOLO.
    
    Args:
        dataset_path (str or Path): Caminho para o diretório raiz do dataset.
    """

    # Iteração pelos Conjuntos de Dados: train (treino), val (validação), test (teste)
    for split in ['train', 'val', 'test']:
        # Carregamento dos dados e verificação de existência.oleta o caminho até o json no formato COCO
        label_path = Path(dataset_path) / "labels" / split / 'coco.json'
        if not label_path.exists():
            logger.warning(f"File not found: {label_path}")
            continue

        with open(label_path) as f:
            data = json.load(f)
        
        # Cria um dicionário que mapeia o ID de cada imagem relacionando com as informações de (nome do arquivo, largura e altura)
        image_info = {img['id']: img for img in data['images']}

        # Processa cada anotação
        for ann in data['annotations']:
            img_id = ann['image_id']
            coco_bbox = ann['bbox']
            category_id = ann['category_id'] - 1  # Assumindo que a categoria ID  Assuming category IDs are 1-indexed in COCO
            img_filename = Path(image_info[img_id]['file_name'])
            img_size = (image_info[img_id]['width'], image_info[img_id]['height'])
            yolo_bbox = convert_coco_to_yolo(img_size, coco_bbox)
            
            txt_path = label_path.parent / (img_filename.stem + '.txt')
            with open(txt_path, 'a') as file:
                file.write(f"{category_id} {' '.join(map(str, yolo_bbox))}\n")
            logger.info(f"Processed annotation for image: {img_filename}")

def create_yaml_file(dataset_path):
    """
    Cria um arquivo .yml para a configuração do dataset utilizando os nomes das classes extraídos de coco.json.
    
    Args:
        dataset_path (str or Path): Caminho para o diretório raiz do dataset.
    """
    label_path = dataset_path / "labels" / 'train' / 'coco.json'
    if not label_path.exists():
        logger.error(f"File not found: {label_path}")
        return

    with open(label_path) as f:
        data = json.load(f)
    class_names = {category['id'] - 1: category['name'] for category in data['categories']}

    # Ensure the classes are sorted by their ids and formatted correctly
    sorted_class_names = sorted(class_names.items())
    class_entries = "\n".join([f"  {id}: {name}" for id, name in sorted_class_names])

    yaml_content = f"""path: {dataset_path.absolute()}  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test:  # test images (optional)

# Classes
names:
{class_entries}
    """

    yaml_path = dataset_path / 'data.yaml'
    with open(yaml_path, 'w') as file:
        file.write(yaml_content.strip())
    logger.info(f"YAML file created at {yaml_path}")

if __name__ == "__main__":
    dataset_path = input("Enter the path to the root directory of the dataset: ")
    dataset_root = Path(dataset_path)
    
    process_annotations(dataset_root)
    create_yaml_file(dataset_root)
